# Ranging Calculations in SpaceLink

This notebook demonstrates the ranging functions available in the SpaceLink library.

In [ ]:
import math

import astropy.units as u
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from spacelink.core import ranging, units

# Set default plot style
plt.rcParams["figure.figsize"] = [12, 8]

## PN Sequence Range Ambiguity

The range ambiguity is the maximum unambiguous range that can be measured with a given PN sequence. It depends on the sequence length and chip rate. The CCSDS and DSN ranging sequences all share the same length of exactly 1,009,470 chips, so the chip rate is the main determiner of the ambiguity distance.

In [ ]:
chip_rates = np.logspace(5, 7, 100) * u.Hz  # From 100 kHz to 10 MHz
range_clock_rates = chip_rates / 2
ambiguities = ranging.pn_sequence_range_ambiguity(range_clock_rates)

plt.figure(figsize=(8, 5))
plt.loglog(chip_rates.to(u.MHz), ambiguities.to(u.km))
plt.xlabel("Chip Rate (MHz)")
plt.ylabel("Range Ambiguity (km)")
plt.title("Range Ambiguity vs. Chip Rate")
plt.grid(True)
plt.show()

## Chip SNR Calculation

The chip SNR determines ranging jitter and acquisition time for PN ranging. It depends on the chip rate and $P_R/N_0$.

In [ ]:
prn0_values = np.arange(30, 70, 0.1) * u.dBHz
range_clock_rate = 1.0 * u.MHz
snr_values = ranging.chip_snr(range_clock_rate, prn0_values)

plt.figure(figsize=(8, 5))
plt.plot(prn0_values, snr_values)
plt.xlabel("$P_R/N_0$ (dBHz)")
plt.ylabel("Chip SNR (dB)")
plt.title(f"Chip SNR vs. $P_R/N_0$ for Range Clock Frequency = {range_clock_rate}")
plt.grid(True)
plt.show()

## Carrier Power Fractions

When a carrier is modulated with both ranging and data signals, the power is distributed among the residual carrier, ranging sidebands, and data sidebands. Let's explore how these power fractions vary with modulation indices.

A couple things to note:

* The ranging and data power fractions shown here are the *usable* power fractions in those sidebands, so the three components do not in general sum to 1. Some power is lost to intermodulation products when both ranging and data are present.
* The functions used in this section apply to the following: 
  * Sinewave ranging clock (sinewave chip pulse shaping for PN ranging).
  * Uplink.
  * Regenerative PN downlink. (These do not apply to the downlink case when a transparent transponder is used.)

First is a plot of the power fractions versus ranging modulation index with the data modulation index held constant.

In [ ]:
ranging_mod_idx = np.linspace(0.0, 1.5, 100) * u.rad
data_mod_idx = 1 / math.sqrt(2) * u.rad
mod_type = ranging.DataModulation.SINE_SUBCARRIER

carrier_power_frac = ranging.carrier_to_total_power(
    ranging_mod_idx, data_mod_idx, mod_type
)
ranging_power = ranging.ranging_to_total_power(ranging_mod_idx, data_mod_idx, mod_type)
data_power_frac = ranging.data_to_total_power(ranging_mod_idx, data_mod_idx, mod_type)
total = carrier_power_frac + ranging_power + data_power_frac

plt.figure()
plt.plot(ranging_mod_idx, carrier_power_frac, label="Residual Carrier Power")
plt.plot(ranging_mod_idx, ranging_power, label="Usable Ranging Power")
plt.plot(ranging_mod_idx, data_power_frac, label="Usable Data Power")
plt.plot(ranging_mod_idx, total, label="Sum")
plt.ylim(0, 1)
plt.xlabel("Ranging Modulation Index (RMS radians)")
plt.ylabel("Power Fraction")
plt.title(
    f"Power Distribution vs. Ranging Modulation Index (Data RMS Mod Index = {data_mod_idx:.3f})"
)
plt.legend()
plt.grid(True)
plt.show()

Next are a set of filled contour plots showing how the power fractions change as a function of both data and ranging modulation indices.

In [ ]:
mod_type = ranging.DataModulation.SINE_SUBCARRIER
mod_idx_vals = np.linspace(0.1, 1.5, 20) * u.rad

mod_idx_ranging, mod_idx_data = np.meshgrid(mod_idx_vals, mod_idx_vals)
carrier_power = ranging.carrier_to_total_power(
    mod_idx_ranging,
    mod_idx_data,
    mod_type,  # type: ignore
)
ranging_power = ranging.ranging_to_total_power(
    mod_idx_ranging,
    mod_idx_data,
    mod_type,  # type: ignore
)
data_power = ranging.data_to_total_power(mod_idx_ranging, mod_idx_data, mod_type)  # type: ignore
total_power = carrier_power + ranging_power + data_power

# Plot power distributions as 2x2 heatmap grid
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Carrier Power (top-left)
im1 = axes[0, 0].contourf(
    mod_idx_ranging,
    mod_idx_data,
    carrier_power.value,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[0, 0].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[0, 0].set_ylabel("Data Modulation Index (RMS rad)")
axes[0, 0].set_title("Residual Carrier Power Fraction")
cbar1 = fig.colorbar(im1, ax=axes[0, 0])
cbar1.set_ticks(np.linspace(0, 1, 6).tolist())

# Ranging Power (top-right)
im2 = axes[0, 1].contourf(
    mod_idx_ranging,
    mod_idx_data,
    ranging_power.value,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[0, 1].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[0, 1].set_ylabel("Data Modulation Index (RMS rad)")
axes[0, 1].set_title("Usable Ranging Power Fraction")
cbar2 = fig.colorbar(im2, ax=axes[0, 1])
cbar2.set_ticks(np.linspace(0, 1, 6).tolist())

# Data Power (bottom-left)
im3 = axes[1, 0].contourf(
    mod_idx_ranging,
    mod_idx_data,
    data_power.value,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[1, 0].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[1, 0].set_ylabel("Data Modulation Index (RMS rad)")
axes[1, 0].set_title("Usable Data Power Fraction")
cbar3 = fig.colorbar(im3, ax=axes[1, 0])
cbar3.set_ticks(np.linspace(0, 1, 6).tolist())

# Total Power (bottom-right)
im4 = axes[1, 1].contourf(
    mod_idx_ranging,
    mod_idx_data,
    total_power.value,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[1, 1].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[1, 1].set_ylabel("Data Modulation Index (RMS rad)")
axes[1, 1].set_title("Sum of Usable and Residual Carrier Powers")
cbar4 = fig.colorbar(im4, ax=axes[1, 1])
cbar4.set_ticks(np.linspace(0, 1, 6).tolist())

plt.tight_layout()
plt.show()

Use the sliders below to interactively explore how the power fractions change with different ranging and data modulation indices.


In [ ]:
def update_power_plot(ranging_mod_idx_val, data_mod_idx_val, modulation_type):
    """Update the power distribution bar plot based on slider values."""

    # Convert slider values to quantities with units
    ranging_mod_idx = ranging_mod_idx_val * u.deg
    data_mod_idx = data_mod_idx_val * u.deg

    # Calculate power fractions
    carrier_power_frac = ranging.carrier_to_total_power(
        ranging_mod_idx, data_mod_idx, modulation_type
    )
    ranging_power_frac = ranging.ranging_to_total_power(
        ranging_mod_idx, data_mod_idx, modulation_type
    )
    data_power_frac = ranging.data_to_total_power(
        ranging_mod_idx, data_mod_idx, modulation_type
    )

    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

    components = [
        "Residual Carrier\nPower",
        "Usable Ranging\nPower",
        "Usable Data\nPower",
    ]
    power_values = [
        carrier_power_frac.value,
        ranging_power_frac.value,
        data_power_frac.value,
    ]
    colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]  # Blue, Orange, Green

    # Left plot: Linear scale
    bars1 = ax1.bar(
        components,
        power_values,
        color=colors,
        alpha=0.8,
        edgecolor="black",
        linewidth=1,
    )

    # Add value labels on top of bars for linear plot
    for bar, value in zip(bars1, power_values, strict=False):
        ax1.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{value:.3f}",
            ha="center",
            va="bottom",
            fontweight="bold",
        )

    ax1.set_ylim(0, 1)
    ax1.set_ylabel("Power Fraction")
    ax1.set_title("Power Fractions")
    ax1.grid(True, alpha=0.3)

    # Add sum information for linear plot
    total_sum = sum(power_values)
    ax1.text(
        0.02,
        0.98,
        f"Sum = {total_sum:.3f}",
        transform=ax1.transAxes,
        bbox=dict(boxstyle="round", facecolor="lightgray", alpha=0.8),
        verticalalignment="top",
        fontweight="bold",
    )

    # Right plot: Decibel scale
    # Convert power fractions to dB using units.to_dB() with default factor=10
    power_values_db = []
    power_values_db_display = []  # For bar heights (finite values only)

    for power_val in power_values:
        if power_val > 0:
            power_db = units.to_dB(power_val * u.dimensionless_unscaled)
            power_values_db.append(power_db.value)
            power_values_db_display.append(power_db.value)
        else:
            power_values_db.append(float("-inf"))  # For labels
            power_values_db_display.append(-50)  # For bar heights (bottom of plot)

    bars2 = ax2.bar(
        components,
        power_values_db_display,
        color=colors,
        alpha=0.8,
        edgecolor="black",
        linewidth=1,
    )

    # Set fixed y-limits for dB plot (power fractions 0-1 map to -∞ to 0 dB)
    ax2.set_ylim(-50, 5)

    # Add value labels on top of bars for dB plot
    for bar, value in zip(bars2, power_values_db, strict=False):
        if value != float("-inf"):
            label_y = (
                bar.get_height() + 0.5
                if bar.get_height() >= 0
                else bar.get_height() - 1.5
            )
            ax2.text(
                bar.get_x() + bar.get_width() / 2,
                label_y,
                f"{value:.1f}",
                ha="center",
                va="bottom" if bar.get_height() >= 0 else "top",
                fontweight="bold",
            )
        else:
            ax2.text(
                bar.get_x() + bar.get_width() / 2,
                -45,
                "-∞",
                ha="center",
                va="center",
                fontweight="bold",
            )

    ax2.set_ylabel("Power Fraction (dB)")
    ax2.grid(True, alpha=0.3)

    # Add sum information for dB plot (convert total sum to dB)
    if total_sum > 0:
        total_sum_db = units.to_dB(total_sum * u.dimensionless_unscaled)
        ax2.text(
            0.02,
            0.98,
            f"Sum = {total_sum_db.value:.1f} dB",
            transform=ax2.transAxes,
            bbox=dict(boxstyle="round", facecolor="lightgray", alpha=0.8),
            verticalalignment="top",
            fontweight="bold",
        )

    # Use manual spacing instead of tight_layout to avoid layout warnings
    plt.subplots_adjust(left=0.08, right=0.95, top=0.85, bottom=0.15, wspace=0.3)
    plt.show()


# Create sliders
ranging_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=180,
    step=1,
    description="Ranging Mod Index (RMS deg):",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
)
data_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=180,
    step=1,
    description="Data Mod Index (RMS deg):",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
)

# Create modulation type selector
modulation_selector = widgets.Dropdown(
    options=[
        ("Bi-polar", ranging.DataModulation.BIPOLAR),
        ("Sinewave Subcarrier", ranging.DataModulation.SINE_SUBCARRIER),
    ],
    value=ranging.DataModulation.BIPOLAR,
    description="Data Modulation Type:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="300px"),
)

# Create interactive widget using widgets.interactive instead of @interact
interactive_plot = widgets.interactive(
    update_power_plot,
    ranging_mod_idx_val=ranging_slider,
    data_mod_idx_val=data_slider,
    modulation_type=modulation_selector,
)

# Display the interactive widget
display(interactive_plot)

## PN Acquisition Probability

An essential function of a PN ranging receiver is acquisition of the ranging code
position, also know as code phase. This operation is what allows the receiver to extend
the range ambiguity from a single range clock period (two chips) out to the full period
of the range code which is 1,009,470 chips.

The acquisition process is fundamentally a detection problem where there is one
correct solution out of 1,009,470 options. One way to quantify the performance of 
acquisition is to predict the probability $P_{\text{acq}}$ that the receiver finds the
correct solution. This is a function of the ranging power to noise density ratio
$P_R/N_0$, the integration time $T$, and the correlation properties of the ranging code.

First we have the probability of acquiring an individual component's code phase. The
ranging receiver solves for each component's code phase separately first, and then uses
all six component solutions to compute the full code phase.

Note how each component's acquisition probability approaches the reciprocal of the 
component length $L_i$ for very low values of $T \cdot P_R/N_0$. For $C_1$ this is 
0.5, and for $C_6$ it's $1/23 \approx 0.043$. Those asymptotes are shown as faint grey
dotted lines.

In [ ]:
# code = ranging.PnRangingCode.CCSDS_T4B
snrs_db = np.linspace(-20, 40, 100) * u.dB
snrs = units.to_linear(snrs_db)
integration_time = 1.0 * u.s

options = [(code.name, code) for code in ranging.PnRangingCode]


@widgets.interact(code=widgets.Dropdown(options=options, description="Code:"))
def foobar(code):
    plt.figure(figsize=(8, 5))
    for component in range(1, 7):
        P_comp_acq = []
        for TPrN0 in snrs:
            P_comp_acq.append(
                ranging.pn_component_acquisition_probability(
                    TPrN0 / integration_time, integration_time, code, component
                )
            )
        plt.axhline(
            1 / ranging.COMPONENT_LENGTHS[component],
            color="grey",
            alpha=0.3,
            linestyle=":",
        )
        plt.plot(snrs_db, P_comp_acq, label=f"$C_{component}$")

    plt.legend(loc="lower right")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.xlabel(r"$T \cdot P_R/N_0$ (dB)")
    plt.ylabel("Acquisition Probability")
    plt.title(f"{code.name} Component Acquisition Probabilities")
    plt.show()

Next we have the full acquisition probability over all code components. This is just
the product of the six component acquisition probabilities since all six must be 
correct.

The differences between the three PN codes are evident. There is a tradeoff between
acquisition performance and range measurement jitter performance, so for example T2B 
requires less SNR for reliable acquisition than T4B, but T4B has better jitter
performance. On the other hand T4B has better acquisition performance than the DSN code
for the same jitter performance.

In [ ]:
snrs_db = np.linspace(10, 40, 100) * u.dB
snrs = units.to_linear(snrs_db)

plt.figure(figsize=(10, 6))

for code in ranging.PnRangingCode:
    P_acq = []
    for snr in snrs:
        integration_time = 1.0 * u.s
        P_acq.append(
            ranging.pn_acquisition_probability(
                snr / integration_time, integration_time, code
            )
        )
    plt.plot(snrs_db, P_acq, label=f"{code.name}")


plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.ylim(0, 1)
plt.xlabel(r"$T \cdot P_R/N_0$ (dB)")
plt.ylabel("Acquisition Probability")
plt.title("Acquisition Probability Comparison")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## PN Acquisition Time

Another way to think about PN ranging acquisition is to calculate how much time is 
required to achieve some desired probability of success at some $P_R/N_0$. Typically a
high probability of success is desired since bad ambiguity resolving (absent other
corrective actions) results in bad range measurements.

In [ ]:
success_probability = 0.999 * u.dimensionless  # 99.9%

# On a log-log plot the trend is linear so only two points are needed. Keeps execution
# time down.
ranging_to_noise_psds = np.array([10, 40]) * u.dBHz

plt.figure(figsize=(10, 6))

for code in ranging.PnRangingCode:
    acq_times = []
    for ranging_to_noise_psd in ranging_to_noise_psds:
        acq_times.append(
            ranging.pn_acquisition_time(
                ranging_to_noise_psd, success_probability, code
            ).value
        )
    plt.semilogy(ranging_to_noise_psds, acq_times, label=f"{code.name}")


plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xlabel(r"$P_R/N_0$ (dBHz)")
plt.ylabel("Acquisition Time (s)")
plt.title("Acquisition Time Comparison for 99.9% Success Probability")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()